# Preview Evaluation

Thia is a notebook to make a preview comparison of multiple inference generation

In [1]:
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
from utils.metrics.calculate_metric import calculate_agg_metric
import os
import numpy as np
import json

/home/caio.rhoden/miniconda3/envs/nq/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 11-20 15:52:51 [__init__.py:216] Automatically detected platform cuda.


In [2]:
exp_names = ["vllm3", "vllm5", "vllm1"]

## Calculate Metrics and save them

In [3]:


questions_path = "../../questions_2500_42_dev.feather"
for exp in exp_names:
    for file in os.listdir(f"../{exp}/generations"):
        if f"result_{exp}.feather" not in os.listdir("."):
            metrics = calculate_agg_metric(
                metrics=["rouge_l"],
                generation_path=f"../{exp}/generations/{file}",
                reference_path=questions_path,
                saving_path=f"result_{exp}.feather"
            )




Could not memory_map compressed IPC file, defaulting to normal read. Toggle off 'memory_map' to silence this warning.
Using the latest cached version of the module from /home/caio.rhoden/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--rouge/b01e0accf3bd6dd24839b769a5fda24e14995071570870922c71970b3a6ed886 (last modified on Wed Oct 16 22:50:24 2024) since it couldn't be found locally at evaluate-metric--rouge, or remotely on the Hugging Face Hub.
Could not memory_map compressed IPC file, defaulting to normal read. Toggle off 'memory_map' to silence this warning.
Could not memory_map compressed IPC file, defaulting to normal read. Toggle off 'memory_map' to silence this warning.


## Load Data

In [4]:
results = []
for f in os.listdir("."):
    if f.endswith("feather"):
        processsed_namefile = f.split(".")[0].split("result_")[1]
        results.append(
            pl.read_ipc(f, memory_map=False)
            .with_columns(
                pl.lit(f"{processsed_namefile}").alias("exp_name"),
            )
        )

results = pl.concat(results)
with pl.Config(
    tbl_rows=-1,             # Show all rows (-1 means all)
    tbl_cols=-1,             # Show all columns (-1 means all)
    fmt_str_lengths=100    # Set string column width
):
    print(results.group_by("exp_name").agg(pl.col("mean").mean()).sort("mean", descending=True))

shape: (3, 2)
┌──────────┬──────────┐
│ exp_name ┆ mean     │
│ ---      ┆ ---      │
│ str      ┆ f64      │
╞══════════╪══════════╡
│ vllm1    ┆ 0.295534 │
│ vllm3    ┆ 0.289639 │
│ vllm5    ┆ 0.286538 │
└──────────┴──────────┘


In [5]:
results.filter(pl.col("mean") > 0).group_by("exp_name").agg(pl.col("mean").count())

exp_name,mean
str,u32
"""vllm1""",1044
"""vllm5""",1281
"""vllm3""",1201


In [6]:
results.filter(pl.col("mean") == 0).group_by("exp_name").agg(pl.col("mean").count())

exp_name,mean
str,u32
"""vllm1""",1456
"""vllm3""",1299
"""vllm5""",1219
